In [1]:
import yfinance as yf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import datetime
import pandas_ta as ta
from tqdm import tqdm
import warnings
import math
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None, 'display.max_columns', None)

In [2]:
import yaml
import random
import datetime
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import torch
import numpy as np
from data_loader_c1 import TwoStreamDataset
from torch.utils.data import DataLoader
from data_loader_c1 import load_dataloader
from model_real_1 import RealCNNLSTM
from model_complex_1 import ComplexCNNLSTM
from model_fusion_1 import FusionGated
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import LambdaLR
from sklearn.metrics import mean_squared_error, mean_absolute_error
from typing import Dict, Tuple, List

In [3]:
def make_results_acc(results_acc, result_tab, config):
    """Запись результатов"""
    results_acc.loc[len(results_acc)] = [result_tab['date'][0], config['data']['time_shift'], config['data']['window_size'], \
    config['data']['lr_lambda'], config['train']['epochs'], config['model']['dropout'], config['model']['n_res_blocks'], \
    round(result_tab['Pred_last_10_ep_mean'].mean(), 4), round(result_tab['Pred_last_10_ep_mean'].std(), 4), \
    round(result_tab['Pred_last_10_ep_std'].mean(), 4), round(result_tab['Pred_last_10_ep_std'].std(), 4)] + \
    result_tab['Pred_last_10_ep_mean'].tolist() + result_tab['Pred_last_10_ep_std'].tolist()
    return results_acc

In [4]:
def custom_time_series_split_4(df_len, time_shift=1, interval_days=1, main_splits_number=2, initial_train_frac=0.90):
    splits = []
    df_len_sh = df_len - interval_days - time_shift + 1
    initial_train_size = int(df_len_sh * initial_train_frac)

    initial_train_size += (df_len_sh - initial_train_size) % main_splits_number
    val_size = int((df_len_sh - initial_train_size) / main_splits_number)
    assert val_size not in range(1)
    
    train_end = initial_train_size - time_shift + 1
    val_start = initial_train_size
    val_end = val_start + val_size

    while val_end <= df_len_sh:
        
        train_idx = list(range(0, train_end))
        val_idx = list(range(val_start, val_end))
        splits.append((train_idx, val_idx))
        train_end = val_end
        val_start = train_end
        val_end = val_start + val_size

    # print(splits)
    one_day_start = splits[-1][1][-1]

    for i in range(interval_days):
        train_idx = list(range(0, one_day_start + i + 1))
        val_idx = list(range(one_day_start + i + time_shift, one_day_start + i + time_shift + 1))
        splits.append((train_idx, val_idx))

    return splits

In [ ]:
# splits = custom_time_series_split_4(55, time_shift=2, interval_days=1, main_splits_number=1, initial_train_frac=0.90)
# splits

In [62]:
# !!! Вариант с двумя фолдами!!!

# ---------- EarlyStopping ----------
class EarlyStopping:
    def __init__(self, patience=10, min_delta=1e-2, mode="min", restore_best=True, rel_delta=True):
        assert mode in ("min", "max")
        self.patience, self.min_delta, self.mode = patience, float(min_delta), mode
        self.restore_best, self.rel_delta = restore_best, rel_delta
        self.best, self.wait, self.best_states = None, 0, {}

    def _improved(self, cur, best):
        if self.mode == "min":
            return cur < (best*(1-self.min_delta) if self.rel_delta else best - self.min_delta)
        else:
            return cur > (best*(1+self.min_delta) if self.rel_delta else best + self.min_delta)

    @torch.no_grad()
    def step(self, metric: float, models: Dict[str, torch.nn.Module]) -> bool:
        if self.best is None or self._improved(metric, self.best):
            self.best, self.wait = metric, 0
            self.best_states = {k:{n:v.detach().cpu().clone() for n,v in m.state_dict().items()}
                                for k,m in models.items()}
            return False
        self.wait += 1
        return self.wait >= self.patience

    @torch.no_grad()
    def restore(self, models: Dict[str, torch.nn.Module]) -> None:
        if not self.best_states: return
        for k,m in models.items():
            if k in self.best_states:
                m.load_state_dict(self.best_states[k])


# ---------- вспомогательные утилиты ----------
def _set_seed(seed=42):
    import os, random
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def _reset_weights(m: nn.Module):
    if isinstance(m, (nn.Conv1d, nn.Linear)):
        m.reset_parameters()
    if isinstance(m, nn.LSTM):
        for name, p in m.named_parameters():
            if 'weight_ih' in name:
                nn.init.xavier_uniform_(p.data)
            elif 'weight_hh' in name:
                nn.init.orthogonal_(p.data)
            elif 'bias' in name:
                nn.init.zeros_(p.data)
                h = p.shape[0]//4
                p.data[h:2*h] = 1.0  # forget-bias


# ---------- основная функция ----------
def train_tscv_model_7(config: dict, df: pd.DataFrame, verbose: bool = True):
    """
    Обучение с кастомным TSCV, EMA-сглаживанием, EarlyStopping, LR-Plateau, grad-accum, clip-grad.
    Возвращает:
        results: dict с метриками по фолдам и сериями предсказаний
    """
    _set_seed(config.get("seed", 42))
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # --- даты и срез df ---
    start_preds_date = pd.to_datetime(config['data']['start_preds_date'], errors='coerce')
    stop_preds_date  = pd.to_datetime(config['data']['stop_preds_date'],  errors='coerce')
    time_shift       = int(config['data']['time_shift'])
    window_size      = int(config['data']['window_size'])
    lr_lambda_type = config['data']['lr_lambda']

    df = df.copy()
    df = df[pd.to_datetime(df['date']) <= stop_preds_date].reset_index(drop=True)

    interval_days = int((stop_preds_date - start_preds_date).days)
    if verbose:
        print("Dates:", start_preds_date, "→", stop_preds_date,
              "| interval_days:", interval_days, "| time_shift:", time_shift)

    # --- сплиты ---
    # splits = custom_time_series_split_3(
    #     len(df),
    #     time_shift=time_shift,
    #     interval_days=interval_days,
    #     main_splits_number=0,
    #     initial_train_frac=0.99
    # )
    # splits = custom_time_series_split_2(len(df), time_shift=time_shift, interval_days=interval_days, main_splits_number=2, initial_train_frac=0.90)
    splits = custom_time_series_split_4(len(df), time_shift=time_shift, interval_days=1, main_splits_number=1, initial_train_frac=0.90)
    if verbose: print('num of splits =', len(splits))

    # --- bootstrap-loader (определяем F и C до инициализации моделей) ---
    first_train_idx, _ = splits[0]
    bootstrap_loader = load_dataloader(
        df.iloc[first_train_idx], window_size=window_size,
        batch_size=config['train']['batch_size'],
        shuffle=False, drop_last=False
    )
    first_batch = next(iter(bootstrap_loader))
    F = first_batch['real_feats'].shape[-1]
    C = (first_batch['complex_time_real'].shape[-1]
         if ('complex_time_real' in first_batch and 'complex_time_imag' in first_batch) else 1)

    # --- модели ---
    real_net = RealCNNLSTM(
        num_real_features=F,
        hidden_dim=config['model']['real_hidden_dim'],
        lstm_layers=config['model'].get('real_lstm_layers', 2),
        dropout=config['model'].get('dropout', 0.3),
        kernel_size=3,
        bidirectional=config['model'].get('bidirectional', True),
        take=config['model'].get('take', "last_timestep"),
        proj_out=True
    ).to(device)

    complex_net = ComplexCNNLSTM(
        in_channels=C,
        hidden_dim=config['model']['complex_hidden_dim'],
        num_layers=config['model']['n_res_blocks'],
        dropout=config['model']['dropout'],
        kernel_size=3,
        proj_out=False
    ).to(device)

    fusion_net = FusionGated(
        real_hidden_dim=config['model']['real_hidden_dim'],
        complex_hidden_dim=config['model']['complex_hidden_dim'],
        output_dim=1, hidden=128, dropout=0.2, use_softmax=True
    ).to(device)


    # --- ✅ Инициализация bias последнего линейного слоя ---
    # with torch.no_grad():
    #     for mod in fusion_net.modules():
    #         if isinstance(mod, torch.nn.Linear) and mod.out_features == 1:
    #             mod.bias.fill_(0.0)   # так как таргет уже нормирован (среднее ~ 0)

    # --- оптимизатор/лоссы/планировщик/ES ---
    optimizer = optim.Adam(
        list(real_net.parameters()) + list(complex_net.parameters()) + list(fusion_net.parameters()),
        lr=config['train']['lr'], weight_decay=1e-4
    )
    criterion = nn.MSELoss() if config['train'].get('loss', 'mse') == 'mse' else nn.SmoothL1Loss(beta=1.0)

    epochs = int(config['train']['epochs'])
    # warmup: 20% эпох, но не менее 1 и не более epochs-1
    warmup_epochs = max(1, min(epochs - 1, int(0.2 * epochs)))
    min_factor = 1e-3

    if lr_lambda_type == 'cos':
        def lr_lambda(e):
            if e < warmup_epochs:
                return (e+1)/warmup_epochs
            # cosine from 1 to ~0
            t = (e - warmup_epochs) / max(1, epochs - warmup_epochs)
            return 0.5*(1 + math.cos(math.pi * t))
    else: 
        def lr_lambda(epoch: int):
            if epoch < epochs // 3:
                return 1.
            elif epochs // 3 <= epoch < (epochs // 3) * 2:
                return 0.5
            else:
                return 0.1

    if lr_lambda_type == 'cos':
        def lr_lambda_fold2(e):
            if e < warmup_epochs:
                return (e+1)/warmup_epochs * 0.2
            # cosine from 1 to ~0
            t = (e - warmup_epochs) / max(1, epochs - warmup_epochs)
            return 0.5*(1 + math.cos(math.pi * t)) * 0.2
    else: 
        def lr_lambda_fold2(epoch: int):
            if epoch < epochs // 3:
                return 0.4
            elif epochs // 3 <= epoch < (epochs // 3) * 2:
                return 0.5 * 0.4
            else:
                return 0.1 * 0.4
            
    # else: 
    #     def lr_lambda(epoch: int):
    #         if epoch < epochs // 4:
    #             return 1.
    #         elif epochs // 4 <= epoch < (epochs // 4) * 2:
    #             return 0.5
    #         elif (epochs // 4) * 2 <= epoch < (epochs // 4) * 3:
    #             return 0.1
    #         else:
    #             return 0.05
            
    scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lr_lambda)

    early = EarlyStopping(
        patience=config['train'].get('early_stop_patience', 10),
        min_delta=config['train'].get('early_stop_min_delta', 1e-2),
        mode="min", restore_best=True, rel_delta=True
    )
    ema_beta = float(config['train'].get('ema_beta', 0.8))

    # --- тренинг-опции ---
    epochs         = int(config['train']['epochs'])
    batch_size     = int(config['train']['batch_size'])
    accum_steps    = int(config['train'].get('accum_steps', 1))
    max_grad_norm  = config['train'].get('max_grad_norm', 1.0)
    min_bs_for_bn  = int(config['train'].get('min_bs_for_bn', 2))
    reset_each_fold= bool(config['train'].get('reset_each_fold', False))

    if verbose: print('size of df:', df.shape)

    # --- нормализация (фиксируем μ,σ по первому train-сплиту) ---
    cols_to_scale = config['data'].get('cols_to_scale', [
        'DayAvgPrice','IntradayStd','Volume','Log_Profit','DayAvgPrice_diff',
        'DAP_1','DAP_2','DAP_3','DAP_4','DAP_5','DAP_6',
        'POLY_1','POLY_2','POLY_3','lambda_C3','lambda_C2','lambda_C1'
    ])

    mu = sigma = None
    mu_target = sigma_target = None

    # --- сбор результатов по фолдам ---
    fold_metrics: List[Dict] = []
    fold_pred_series: List[pd.Series] = []
    preds_last_10 = []

    for fold, (train_idx, val_idx) in enumerate(splits):
        # опциональный ресет весов (для «честной» CV)
        if reset_each_fold and (fold + 1) > 1:
            # real_net.apply(_reset_weights)
            # complex_net.apply(_reset_weights)
            # fusion_net.apply(_reset_weights)
            # optimizer = optim.Adam(
            #     list(real_net.parameters()) + list(complex_net.parameters()) + list(fusion_net.parameters()),
            #     lr=config['train']['lr']
            # )
            # сбросить состояние LR-схем, ES, EMA
            # scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            #     optimizer, mode='min', factor=0.5, patience=100, threshold=1e-3, threshold_mode='rel',
            #     cooldown=2, verbose=True
            # )
            # scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            #     optimizer, mode='min', factor=0.5, patience=5, verbose=verbose
            # )
            scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lr_lambda_fold2)
            early = EarlyStopping(
                patience=config['train'].get('early_stop_patience', 10),
                min_delta=config['train'].get('early_stop_min_delta', 1e-2),
                mode="min", restore_best=True, rel_delta=True
            )
            ema_val = None
        else:
            ema_val = None  # сбрасываем EMA в любом случае на новый фолд (иначе метрики не сопоставимы)

        train_df = df.iloc[train_idx].copy()

        # расширим val назад до window_size, безопасно по границам
        if len(val_idx) <= window_size:
            end = val_idx[-1]
            start = max(0, end - window_size)
            val_idx = list(range(start, end + 1))

        val_df = df.iloc[val_idx].copy()

        # μ,σ считаем по ПЕРВОМУ train-сплиту
        if mu is None or sigma is None:
            mu = train_df[cols_to_scale].mean()
            sigma = train_df[cols_to_scale].std().replace(0, 1.0)

        # применяем нормировку
        for col in cols_to_scale:
            train_df.loc[:, col] = (train_df[col] - mu[col]) / sigma[col]
            val_df.loc[:,   col] = (val_df[col]   - mu[col]) / sigma[col]

        # μ,σ для z-нормировки Target
        if mu_target is None or sigma_target is None:
            mu_target = train_df['Target'].mean()
            sigma_target = train_df['Target'].std()

        train_df['Target'] = (train_df['Target'] - mu_target) / sigma_target
        val_df['Target'] = (val_df['Target']   - mu_target) / sigma_target 
        train_df['Target_smooth'] = (train_df['Target_smooth'] - mu_target) / sigma_target
        val_df['Target_smooth'] = (val_df['Target_smooth']   - mu_target) / sigma_target  

        #print(f"\ntrain_end {train_df.iloc[-1]}, val_end {val_df.iloc[-1]}")  

        # лоадеры
        train_loader = load_dataloader(train_df, window_size, batch_size, shuffle=True,  drop_last=False)
        val_loader   = load_dataloader(val_df,   window_size, batch_size, shuffle=False, drop_last=False)

        if verbose:
            print(f"\nFold {fold+1}: train {len(train_df)}, val {len(val_df)}",
                  "\ntrain first...last :", train_df['date'].min(), "..." , train_df['date'].max(),
                  "\nval first...last   :", val_df['date'].min(), "..." ,   val_df['date'].max())

        # контрольный принт последнего окна
            
        dst   = TwoStreamDataset(train_df, window_size=window_size)
        sampt = dst[len(dst) - 1]
        last_DAP_in_t_window = float(sampt['complex_time_real'][-1, 1]) * sigma['c_month_real'] + mu['c_month_real']
        target_pricet           = float(sampt['target']) * sigma_target + mu_target
        target_smooth_pricet = train_df['Target_smooth'].iloc[-1] * sigma_target + mu_target
        if verbose:
            print(f"train last row | DAP={last_DAP_in_t_window:.4f} | Target={target_pricet:.4f} | Target_smooth={target_smooth_pricet:.4f}")

        ds   = TwoStreamDataset(val_df, window_size=window_size)
        samp = ds[len(ds) - 1]
        # last_DAP_in_val_window = float(samp['real_feats'][-1, 0]) * sigma['DayAvgPrice'] + mu['DayAvgPrice']
        last_DAP_in_val_window = float(samp['complex_time_real'][-1, 1]) * sigma['c_month_real'] + mu['c_month_real']
        # last_DAP_in_val_window = val_df['imag_time'].iloc[-1] * sigma['imag_time'] + mu['imag_time']
        # last_DAP_in_val_window = val_df['DayAvgPrice'].iloc[-1] * sigma['DayAvgPrice'] + mu['DayAvgPrice']
        target_price           = float(samp['target']) * sigma_target + mu_target
        target_smooth_price = val_df['Target_smooth'].iloc[-1] * sigma_target + mu_target
        if verbose:
            print(f"val last row   | DAP={last_DAP_in_val_window:.4f} | Target={target_price:.4f} | Target_smooth={target_smooth_price:.4f}")

        # Графики
        # from utils_live_plot import EMAMeter, TrainingPlotter  # если вынес в файл, иначе опусти импорт
        # ema_meter    = EMAMeter(beta=config['train'].get('ema_beta', 0.8))
        # live_plotter = TrainingPlotter(max_points=500)

        # ----- ЭПОХИ -----
        for epoch in range(epochs):
            real_net.train(); complex_net.train(); fusion_net.train()
            optimizer.zero_grad(set_to_none=True)

            # TRAIN
            step = 0
            for step, batch in enumerate(train_loader, start=1):
                x_real = batch['real_feats'].to(device)
                if x_real.size(0) < min_bs_for_bn:
                    continue
                x_complex = torch.complex(
                    batch['complex_time_real'].to(device),
                    batch['complex_time_imag'].to(device)
                )
                y = batch['target'].to(device).unsqueeze(-1)

                h_real = real_net(x_real)
                h_r, h_i = complex_net(x_complex)
                out = fusion_net(h_real, h_r, h_i)

                loss = criterion(out, y) / accum_steps
                if step == len(train_loader) - 1: train_loss_temp = loss
                loss.backward()

                if step % accum_steps == 0:
                    if max_grad_norm is not None:
                        torch.nn.utils.clip_grad_norm_(
                            list(real_net.parameters()) + list(complex_net.parameters()) + list(fusion_net.parameters()),
                            max_grad_norm
                        )
                    optimizer.step()
                    optimizer.zero_grad(set_to_none=True)

            # доводим хвост аккумулирования
            if step % accum_steps != 0:
                if max_grad_norm is not None:
                    torch.nn.utils.clip_grad_norm_(
                        list(real_net.parameters()) + list(complex_net.parameters()) + list(fusion_net.parameters()),
                        max_grad_norm
                    )
                optimizer.step()
                optimizer.zero_grad(set_to_none=True)

            # VALID
            real_net.eval(); complex_net.eval(); fusion_net.eval()
            val_loss_sum, n_val = 0.0, 0
            pred_vals, pred_dates = [], []
            with torch.no_grad():
                for batch in val_loader:
                    x_real = batch['real_feats'].to(device)
                    x_complex = torch.complex(
                        batch['complex_time_real'].to(device),
                        batch['complex_time_imag'].to(device)
                    )
                    y = batch['target'].to(device).unsqueeze(-1)

                    h_real = real_net(x_real)
                    h_r, h_i = complex_net(x_complex)
                    out = fusion_net(h_real, h_r, h_i)

                    val_loss_sum += criterion(out, y).item()
                    n_val += 1

                    # даты предиктов
                    pred_vals.extend(out.squeeze(-1).cpu().numpy())

                    if 'target_date' in batch:
                        pred_dates.extend(list(batch['target_date']))
                    else:
                        # fallback: считаем по глобальному индексу
                        batch_idx_local  = batch['row_idx'].cpu().numpy()   # индекс окна в val_df
                        global_start     = val_idx[0]                        # смещение в исходном df
                        batch_idx_global = global_start + batch_idx_local
                        # y = target[idx + window_size] → цель на "завтра" относительно начала окна
                        tgt_idx = batch_idx_global + window_size
                        pred_dates.extend(df['date'].iloc[tgt_idx].tolist())

            avg_val_loss = val_loss_sum / max(1, n_val)

            # --- EMA для scheduler/early ---
            ema_val = avg_val_loss if (ema_val is None) else (ema_beta*ema_val + (1-ema_beta)*avg_val_loss)
            use_metric = ema_val

            scheduler.step()
            # if epoch+1 >= config['train'].get('early_stop_warmup', 0):
            #     scheduler.step()
            #     #scheduler.step(use_metric)
            #     stop = early.step(use_metric, {"real": real_net, "complex": complex_net, "fusion": fusion_net})
            #     # диагностика (разово/по условию)
            #     # print(f"[ES] e{epoch+1} metric={use_metric:.5f}, best={early.best}, wait={early.wait}")
            #     if stop:
            #         if verbose:
            #             print(f"[EarlyStop] best(ema)={early.best:.4f} → restore best weights")
            #         early.restore({"real": real_net, "complex": complex_net, "fusion": fusion_net})
            #         break
            # else:
            #     pass

            # EMA для scheduler/early
            # ema_val = avg_val_loss if (ema_val is None) else (ema_beta*ema_val + (1-ema_beta)*avg_val_loss)
            # use_metric = ema_val  # <— ключевая строка: ES и Plateau смотрят на одно и то же

            # Графики
            # если ты уже считаешь ema_val сам — можно синхронизировать:
            # ema_val = ema_meter.update(avg_val_loss)
            # # обновляем график каждые, скажем, 1-5 эпох
            # live_plotter.update(epoch+1, avg_val_loss, ema_val)
            # if (epoch+1) % 1 == 0:   # частота обновления
            #     live_plotter.show(title=f"Fold {fold+1} — Validation Loss vs EMA")
            # Графики
            # pred_val_denorm = [x * sigma_target + mu_target for x in pred_vals][0]    # откат нормировки целевой переменной
            # live_plotter.update(epoch+1, pred_val_denorm, last_DAP_in_val_window)
            # if (epoch+1) % 1 == 0:   # частота обновления
            #     live_plotter.show(title=f"Fold {fold+1} — Predict vs DAP")
            # Графики

            # scheduler.step(use_metric)
            # stop = early.step(use_metric, {"real": real_net, "complex": complex_net, "fusion": fusion_net})
            pred_vals_denorm = [x * sigma_target + mu_target for x in pred_vals]    # откат нормировки целевой переменной
            if verbose and ((epoch+1) % 4 == 0 or (epoch+1) == 1):
                print(f"Epoch {epoch+1:03d} | train_loss={train_loss_temp:.6f} | val_loss={avg_val_loss:.4f} | ema={ema_val:.4f} | lr={optimizer.param_groups[0]['lr']:.5f} | pred = {pred_vals_denorm[0]:.4f}")

            if (epoch >= (epochs - 10)) and (fold + 1 == len(splits)):
                preds_last_10.append(round(pred_vals_denorm[0], 4))
            # if stop:
            #     if verbose:
            #         print(f"[EarlyStop] best(ema)={early.best:.6f} → restore best weights")
            #     early.restore({"real": real_net, "complex": complex_net, "fusion": fusion_net})
            #     break

        # пост-валид метрики по датам target
        pred_vals_denorm = [x * sigma_target + mu_target for x in pred_vals]    # откат нормировки целевой переменной
        print(f"pred = {pred_vals_denorm[-1]:.4f}, {len(pred_vals_denorm)}")
        pred_series = pd.Series(pred_vals_denorm, index=pd.to_datetime(pred_dates))
        gt_series   = pd.to_datetime(df['date']).map(pd.Timestamp).map(
            lambda d: d
        )  # просто для явности типов
        # берём фактические target по тем же датам:
        gt_series = df.set_index(pd.to_datetime(df['date']))['Target'].reindex(pred_series.index)

        mae_target_pred = mean_absolute_error(gt_series, pred_series)
        # dap_series = df.set_index(pd.to_datetime(df['date']))['DayAvgPrice'].reindex(pred_series.index)
        dap_series = df.set_index(pd.to_datetime(df['date']))['c_month_real'].reindex(pred_series.index)
        mae_target_dap  = mean_absolute_error(gt_series, dap_series)

        if fold + 1 == len(splits):
            fold_pred_series.append(pred_series)
            fold_metrics.append({
                "fold": fold + 1,
                "n_pred": len(pred_series),
                "mae_target_pred": float(mae_target_pred),
                "mae_target_dap":  float(mae_target_dap),
                "val_loss_last_epoch": float(avg_val_loss),
            })

        if verbose:
            print(f"[Fold {fold+1}] Target = {gt_series[-1]:.4f} | Predict = {pred_series[-1]:.4f}")
            print(f"MAE(Target~DAP) = {mae_target_dap:.4f} | MAE(Target~Predict) = {mae_target_pred:.4f}\n")

    torch.save(real_net.state_dict(), f"../checkpoints/trained_real_net_{config['train']['lr']}.pth")
    torch.save(complex_net.state_dict(), f"../checkpoints/trained_complex_net_{config['train']['lr']}.pth")
    torch.save(fusion_net.state_dict(), f"../checkpoints/trained_fusion_net_{config['train']['lr']}.pth")

    # агрегированные результаты
    results = {
        "fold_metrics": fold_metrics,
        "pred_series_per_fold": fold_pred_series,
        "preds_last_10": preds_last_10,
        "mu": mu, "sigma": sigma,
    }
    return results

In [6]:
cols_to_scale = ['open', 'high', 'low', 'close', 'DayAvgPrice', 'IntradayStd',
       'Volume', 'day_of_week', 'day_of_year', 'Log_Profit',
       'DayAvgPrice_diff', 'DayAvgPrice_2diff', 'POLY_1', 'POLY_2', 'POLY_3',
       'parkinson_vol', 'parkinson_vol_ma5',
       'parkinson_vol_ma20', 'parkinson_vol_diff1', 'parkinson_vol_lag1',
       'DAP_1', 'DAP_2', 'DAP_3', 'DAP_4', 'DAP_5', 'DAP_6', 'DAP_7', 'DAP_10', 
       'DayAvgPrice_roll5', 'DayAvgPrice_roll10', 'DayAvgPrice_roll20',
       'DayAvgPrice_ema5', 'DayAvgPrice_ema20', 'IntradayStd_roll5',
       'IntradayStd_roll10', 'IntradayStd_roll20', 'IntradayStd_ema5',
       'IntradayStd_ema20', 'mean_w', 'std_w',
       'q10_w', 'q90_w', 'slope_w', 'vol_w', 'macd', 'macd_signal', 'macd_hist',
       'bb_low', 'bb_mid', 'bb_up', 'atr14', 'dphi_hilbert',
       'stft_energy_low', 'stft_energy_mid', 'stft_energy_high',
       'gk_sigma', 'rs_sigma', 'yz_sigma', 'adx', 'chop', 'kalm_slope', 'rv20', 'vol_of_vol', 
       'bb_pct_b', 'bb_bandwidth', 'ret_overnight', 'ret_intraday', 'corr_ret_dlogvol', 
       'c_month_real',
       ]

In [71]:
config = {
  "seed": 42,
  "data": {
    "start_preds_date": "2025-11-11",
    "stop_preds_date": "2025-11-12",
    "time_shift": 2,
    "window_size": 21,
    "cols_to_scale": cols_to_scale,
    "lr_lambda": "cos"
  },
  "model": {
    "real_hidden_dim": 128,
    "complex_hidden_dim": 96,
    "n_res_blocks": 2,
    "dropout": 0.4,
    "bidirectional": True,
    "take": "last_timestep",
    "real_lstm_layers": 2
  },
  "train": {
    "lr": 0.001,
    "loss": "SmoothL1Loss",
    "epochs": 64,
    "batch_size": 96,
    "accum_steps": 1,
    "max_grad_norm": 1.0,
    "min_bs_for_bn": 2,
    "early_stop_patience": 200,
    "early_stop_min_delta": 1e-2,
    "ema_beta": 0.8,
    "reset_each_fold": True
  }
}

In [8]:
# Загружаем конфигурацию
# df = pd.read_csv('../data_archiv/DTG/DTG_new_fea_to_12_11_2025_1d_w21_noweekend.csv')

In [72]:
# Загружаем конфигурацию
df = pd.read_csv(f"../data_archiv/DTG/DTG_new_fea_to_12_11_2025_1d_w{config['data']['window_size']}_noweekend.csv")

In [73]:
# Создаём Target, пустые последние значения заменяются 0.
H = config['data']['time_shift']     # time shift
# df['Target'] = df['DayAvgPrice'].shift( - H).fillna(method='ffill')
# df['Target'].iloc[-H:] = 0
df['Target'] = df['DayAvgPrice'].shift( - H).fillna(0)
df[['date', 'DayAvgPrice', 'Target']].tail(15)

,date,DayAvgPrice,Target
916,2025-10-23,35.070833,35.094444
917,2025-10-24,35.294445,34.881389
918,2025-10-27,35.094444,35.007222
919,2025-10-28,34.881389,34.865000
920,2025-10-29,35.007222,34.751945
921,2025-10-30,34.865000,34.606944
922,2025-10-31,34.751945,33.886111
923,2025-11-03,34.606944,34.891389
924,2025-11-04,33.886111,35.163611
925,2025-11-05,34.891389,34.328333


In [74]:
df['c_month_real'] = df['DayAvgPrice'].copy()
df['c_month_imag'] = df['IntradayStd'].copy()
df['DayAvgPrice'] = df['c_week_real'].copy()
df['IntradayStd'] = df['c_week_imag'].copy()

In [75]:
# Lr_list predict: START! 

start_preds_date = pd.to_datetime(config['data']['start_preds_date'], errors='coerce')
stop_preds_date = pd.to_datetime(config['data']['stop_preds_date'], errors='coerce')
interval_days = int((stop_preds_date - start_preds_date).days)
start_index = df.index[df['date']==config['data']['start_preds_date']].tolist()[0]
result_list = []

cum_mae_pred = 0.0
cum_mae_dap  = 0.0
steps = 0

start_lr_list = [0.003, 0.002, 0.001, 0.0009, 0.0008, 0.0007, 0.0006, 0.0005, 0.0004, 0.0003]
results_acc = pd.read_csv('../data_archiv/DTG/res/DTG_res_fresh.csv')
total_iters = len(start_lr_list)

with tqdm(total=total_iters, desc="Rolling forecast") as pbar:
    for start_lr in start_lr_list:
        df_temp = df.copy()

        config['train']['lr'] = start_lr

        results = train_tscv_model_7(config, df_temp, verbose=True)

        DAP_ = round(float(df['c_month_real'].iloc[start_index + 1]), 4)
        mu_DAP = round(float(results["mu"]['c_month_real']), 4)
        sigma_DAP = round(float(results["sigma"]['c_month_real']), 4)
        # DAP_ = DAP * sigma_DAP + mu_DAP

        Target_ = round(float(df_temp['Target'].iloc[start_index + 1]), 4)
        Pred = round(float(results["pred_series_per_fold"][0]), 4)
        preds_last_10_mean = round(pd.Series(results["preds_last_10"]).mean(), 4)
        preds_last_10_std = round(pd.Series(results["preds_last_10"]).std(), 4)
        print(results["preds_last_10"], f" | mean={preds_last_10_mean:.4f} | std={preds_last_10_std:.4f}\n")

        mae_target_pred = results["fold_metrics"][-1]["mae_target_pred"]
        mae_target_dap = abs(DAP_ - Target_)
        mae_target_pred_avg = abs(preds_last_10_mean - Target_)
        # mae_target_dap = results["fold_metrics"][-1]["mae_target_dap"]

        result_list.append({
            'date': config['data']['stop_preds_date'],
            'lr': config['train']['lr'],
            'last_DAP': DAP_,
            'mu_DAP': round(float(results["mu"]['c_month_real']), 4),
            'Target': Target_,
            'Predict': Pred,
            'Pred_last_10_ep_mean': preds_last_10_mean,
            'Pred_last_10_ep_std': preds_last_10_std,
            'mae_target_pred': round(mae_target_pred, 4),
            'mae_target_pred_avg': round(mae_target_pred_avg, 4),
            'mae_target_dap': round(mae_target_dap, 4)
        })

        # обновляем кумулятивные средние
        steps += 1
        cum_mae_pred = ((cum_mae_pred * (steps - 1)) + mae_target_pred) / steps
        cum_mae_dap  = ((cum_mae_dap  * (steps - 1)) + mae_target_dap)  / steps

        # обновляем прогресс-бар: последние и средние MAE
        pbar.set_postfix({
            "last_mae_pred": f"{mae_target_pred:.4f}",
            "last_mae_dap":  f"{mae_target_dap:.4f}",
            "avg_mae_pred":  f"{cum_mae_pred:.4f}",
            "avg_mae_dap":   f"{cum_mae_dap:.4f}"
        })
        pbar.update(1)

# собираем итоговый датафрейм один раз
result_tab = pd.DataFrame(result_list)
make_results_acc(results_acc, result_tab, config)
results_acc.to_csv('../data_archiv/DTG/res/DTG_res_fresh.csv', index=False)
display(results_acc)

Rolling forecast:   0%|          | 0/10 [00:00<?, ?it/s]

Dates: 2025-11-11 00:00:00 → 2025-11-12 00:00:00 | interval_days: 1 | time_shift: 2
num of splits = 2
size of df: (931, 107)

Fold 1: train 835, val 93 
train first...last : 2022-03-23 ... 2025-07-01 
val first...last   : 2025-07-03 ... 2025-11-10
train last row | DAP=39.4764 | Target=40.7472 | Target_smooth=39.5991
val last row   | DAP=35.7539 | Target=36.2994 | Target_smooth=35.3047
Epoch 001 | train_loss=0.446030 | val_loss=0.1710 | ema=0.1710 | lr=0.00050 | pred = 35.5744
Epoch 004 | train_loss=0.065899 | val_loss=0.1196 | ema=0.1418 | lr=0.00125 | pred = 43.6171
Epoch 008 | train_loss=0.097971 | val_loss=0.0143 | ema=0.0775 | lr=0.00225 | pred = 41.3635
Epoch 012 | train_loss=0.100443 | val_loss=0.0819 | ema=0.0635 | lr=0.00300 | pred = 40.9599
Epoch 016 | train_loss=0.026600 | val_loss=0.0217 | ema=0.0417 | lr=0.00296 | pred = 41.1615
Epoch 020 | train_loss=0.031063 | val_loss=0.0304 | ema=0.0333 | lr=0.00283 | pred = 38.8893
Epoch 024 | train_loss=0.027910 | val_loss=0.0686 | em

Rolling forecast:  10%|█         | 1/10 [06:47<1:01:09, 407.78s/it, last_mae_pred=36.4423, last_mae_dap=36.2994, avg_mae_pred=36.4423, avg_mae_dap=36.2994]

Epoch 064 | train_loss=0.011616 | val_loss=6.2591 | ema=6.2563 | lr=0.00000 | pred = 36.4423
pred = 36.4423, 1
[Fold 2] Target = 0.0000 | Predict = 36.4423
MAE(Target~DAP) = 36.2994 | MAE(Target~Predict) = 36.4423

[36.4461, 36.4952, 36.4104, 36.3961, 36.3912, 36.4135, 36.4369, 36.4444, 36.4424, 36.4423]  | mean=36.4319 | std=0.0305

Dates: 2025-11-11 00:00:00 → 2025-11-12 00:00:00 | interval_days: 1 | time_shift: 2
num of splits = 2
size of df: (931, 107)

Fold 1: train 835, val 93 
train first...last : 2022-03-23 ... 2025-07-01 
val first...last   : 2025-07-03 ... 2025-11-10
train last row | DAP=39.4764 | Target=40.7472 | Target_smooth=39.5991
val last row   | DAP=35.7539 | Target=36.2994 | Target_smooth=35.3047
Epoch 001 | train_loss=0.466154 | val_loss=0.2850 | ema=0.2850 | lr=0.00033 | pred = 34.5400
Epoch 004 | train_loss=0.113900 | val_loss=0.1527 | ema=0.2110 | lr=0.00083 | pred = 41.2000
Epoch 008 | train_loss=0.153910 | val_loss=0.0520 | ema=0.1376 | lr=0.00150 | pred = 43.79

Rolling forecast:  20%|██        | 2/10 [13:43<54:58, 412.29s/it, last_mae_pred=36.3372, last_mae_dap=36.2994, avg_mae_pred=36.3898, avg_mae_dap=36.2994]  

...

Rolling forecast:  90%|█████████ | 9/10 [1:01:08<06:43, 403.95s/it, last_mae_pred=36.9453, last_mae_dap=36.2994, avg_mae_pred=36.6706, avg_mae_dap=36.2994]

Epoch 064 | train_loss=0.018663 | val_loss=6.3524 | ema=6.3533 | lr=0.00000 | pred = 36.9453
pred = 36.9453, 1
[Fold 2] Target = 0.0000 | Predict = 36.9453
MAE(Target~DAP) = 36.2994 | MAE(Target~Predict) = 36.9453

[36.9137, 36.985, 37.0252, 36.9929, 36.9579, 36.9392, 36.9428, 36.9455, 36.9451, 36.9453]  | mean=36.9593 | std=0.0325

Dates: 2025-11-11 00:00:00 → 2025-11-12 00:00:00 | interval_days: 1 | time_shift: 2
num of splits = 2
size of df: (931, 107)

Fold 1: train 835, val 93 
train first...last : 2022-03-23 ... 2025-07-01 
val first...last   : 2025-07-03 ... 2025-11-10
train last row | DAP=39.4764 | Target=40.7472 | Target_smooth=39.5991
val last row   | DAP=35.7539 | Target=36.2994 | Target_smooth=35.3047
Epoch 001 | train_loss=0.499901 | val_loss=0.3166 | ema=0.3166 | lr=0.00005 | pred = 33.9467
Epoch 004 | train_loss=0.231702 | val_loss=0.1074 | ema=0.2449 | lr=0.00013 | pred = 37.0653
Epoch 008 | train_loss=0.148661 | val_loss=0.0922 | ema=0.1680 | lr=0.00022 | pred = 41.833

Rolling forecast: 100%|██████████| 10/10 [1:07:49<00:00, 406.96s/it, last_mae_pred=36.3090, last_mae_dap=36.2994, avg_mae_pred=36.6344, avg_mae_dap=36.2994]

Epoch 064 | train_loss=0.020220 | val_loss=6.2343 | ema=6.2380 | lr=0.00000 | pred = 36.3090
pred = 36.3090, 1
[Fold 2] Target = 0.0000 | Predict = 36.3090
MAE(Target~DAP) = 36.2994 | MAE(Target~Predict) = 36.3090

[36.3675, 36.4154, 36.4404, 36.3996, 36.3399, 36.3122, 36.311, 36.309, 36.3079, 36.309]  | mean=36.3512 | std=0.0510



,date,time_shift,window_size,lr_lambda,epochs,dropout,n_res_blocks,PL10_mean_mean,PL10_mean_std,PL10_std_mean,PL10_std_std,0.0030_mean,0.0020_mean,0.0010_mean,0.0009_mean,0.0008_mean,0.0007_mean,0.0006_mean,0.0005_mean,0.0004_mean,0.0003_mean,0.0030_std,0.0020_std,0.0010_std,0.0009_std,0.0008_std,0.0007_std,0.0006_std,0.0005_std,0.0004_std,0.0003_std
0,2025-11-09,3,92,cos,44,0.3,3,34.04296,0.296592,0.03677,0.032337,33.8967,33.7666,34.1048,34.4722,34.3281,34.1199,34.1343,33.9614,34.2144,33.4312,0.0230,0.0407,0.0196,0.0111,0.0304,0.0233,0.0211,0.0476,0.0271,0.1238
1,2025-11-09,2,20,cos,52,0.3,3,34.08798,0.101572,0.02693,0.010317,33.9794,34.2996,34.1232,34.1260,34.0834,34.0315,34.0361,34.1098,34.1540,33.9368,0.0222,0.0314,0.0139,0.0264,0.0359,0.0211,0.0506,0.0239,0.0195,0.0244
2,2025-11-09,4,61,cos,36,0.3,1,34.52822,0.428319,0.07696,0.025820,34.8397,35.0390,35.1257,34.1310,34.6296,34.7337,34.0210,34.6563,34.0728,34.0334,0.1019,0.0576,0.1342,0.0958,0.0539,0.0594,0.0603,0.0749,0.0652,0.0664
3,2025-11-09,5,92,cos,36,0.3,3,33.41389,0.592093,0.13273,0.085554,33.9495,34.1444,33.5586,33.4852,32.9203,34.1612,33.1922,33.3552,33.1304,32.2419,0.0545,0.0252,0.0810,0.0973,0.1441,0.1443,0.1279,0.3019,0.2520,0.0991
4,2025-11-09,2,61,_cos,40,0.3,2,34.01628,0.334351,0.07227,0.023782,34.3612,34.4437,34.0495,33.9382,33.3032,34.1230,33.9392,33.9461,33.7411,34.3176,0.0815,0.0663,0.0519,0.0770,0.1233,0.0834,0.0848,0.0614,0.0567,0.0364
5,2025-11-09,1,61,_cos,40,0.4,2,34.09213,0.271552,0.04879,0.013215,34.5063,34.4195,34.0148,33.8809,33.9621,33.8735,33.8598,33.7669,34.3787,34.2588,0.0596,0.0394,0.0345,0.0579,0.0617,0.0406,0.0459,0.0678,0.0274,0.0531
6,2025-11-07,1,20,cos,48,0.4,2,34.03229,0.147213,0.04455,0.014940,33.8739,33.8561,34.0091,34.1713,34.1532,33.8691,34.1332,33.9426,34.0381,34.2763,0.0277,0.0379,0.0543,0.0614,0.0696,0.0501,0.0495,0.0227,0.0357,0.0366
7,2025-11-07,2,34,cos,48,0.4,2,34.20160,0.203500,0.03720,0.024600,33.8877,33.8170,34.3975,34.1070,34.3238,34.2922,34.3360,34.2902,34.3743,34.1906,0.0247,0.0238,0.0367,0.0367,0.0163,0.1037,0.0428,0.0314,0.0265,0.0291
8,2025-11-07,3,34,cos,48,0.4,2,33.77220,0.147600,0.09480,0.032400,33.7382,33.8192,33.5882,33.7218,33.4791,33.9450,33.9421,33.8468,33.8459,33.7960,0.1201,0.0930,0.0927,0.0939,0.0571,0.1116,0.1641,0.0793,0.0508,0.0850
9,2025-11-07,2,34,cos,60,0.3,3,34.16810,0.252500,0.05370,0.024600,33.9715,34.0325,34.5261,34.3204,34.4775,34.1714,34.3446,33.7550,33.9080,34.1736,0.0252,0.0654,0.0168,0.0602,0.0510,0.0906,0.0771,0.0743,0.0499,0.0266


In [76]:
result_tab

,date,lr,last_DAP,mu_DAP,Target,Predict,Pred_last_10_ep_mean,Pred_last_10_ep_std,mae_target_pred,mae_target_pred_avg,mae_target_dap
0,2025-11-12,0.0030,36.2994,33.1234,0.0,36.4423,36.4319,0.0305,36.4423,36.4319,36.2994
1,2025-11-12,0.0020,36.2994,33.1234,0.0,36.3372,36.3303,0.0271,36.3372,36.3303,36.2994
2,2025-11-12,0.0010,36.2994,33.1234,0.0,36.3900,36.3830,0.0450,36.3900,36.3830,36.2994
3,2025-11-12,0.0009,36.2994,33.1234,0.0,36.8480,36.8548,0.0486,36.8480,36.8548,36.2994
4,2025-11-12,0.0008,36.2994,33.1234,0.0,36.7379,36.7580,0.0504,36.7379,36.7580,36.2994
5,2025-11-12,0.0007,36.2994,33.1234,0.0,36.6947,36.6907,0.0256,36.6947,36.6907,36.2994
6,2025-11-12,0.0006,36.2994,33.1234,0.0,36.6517,36.6623,0.0393,36.6517,36.6623,36.2994
7,2025-11-12,0.0005,36.2994,33.1234,0.0,36.9879,37.0027,0.0193,36.9879,37.0027,36.2994
8,2025-11-12,0.0004,36.2994,33.1234,0.0,36.9453,36.9593,0.0325,36.9453,36.9593,36.2994
9,2025-11-12,0.0003,36.2994,33.1234,0.0,36.3090,36.3512,0.0510,36.3090,36.3512,36.2994


In [77]:
print(round(result_tab['Pred_last_10_ep_mean'].mean(), 4), round(result_tab['Pred_last_10_ep_mean'].std(), 4), \
      round(result_tab['Pred_last_10_ep_std'].mean(), 4))
my_list = result_tab['Pred_last_10_ep_mean'].tolist()
my_list.remove(max(my_list))
my_list.remove(min(my_list))
sum(my_list) / len(my_list)

36.6424 0.255 0.0369


36.636399999999995

In [64]:
# result_tab['Predict_mean_10_last_epochs'] = result_tab['Pred_last_10_ep_mean'].copy()
# result_tab['Predict_std_10_last_epochs'] = result_tab['Pred_last_10_ep_std'].copy()

In [ ]:
# result_tab[['lr', 'Predict_mean_10_last_epochs', 'Predict_std_10_last_epochs']]

In [59]:
results_acc

,date,time_shift,window_size,lr_lambda,epochs,dropout,n_res_blocks,PL10_mean_mean,PL10_mean_std,PL10_std_mean,PL10_std_std,0.0030_mean,0.0020_mean,0.0010_mean,0.0009_mean,0.0008_mean,0.0007_mean,0.0006_mean,0.0005_mean,0.0004_mean,0.0003_mean,0.0030_std,0.0020_std,0.0010_std,0.0009_std,0.0008_std,0.0007_std,0.0006_std,0.0005_std,0.0004_std,0.0003_std
0,2025-11-09,3,92,cos,44,0.3,3,34.04296,0.296592,0.03677,0.032337,33.8967,33.7666,34.1048,34.4722,34.3281,34.1199,34.1343,33.9614,34.2144,33.4312,0.0230,0.0407,0.0196,0.0111,0.0304,0.0233,0.0211,0.0476,0.0271,0.1238
1,2025-11-09,2,20,cos,52,0.3,3,34.08798,0.101572,0.02693,0.010317,33.9794,34.2996,34.1232,34.1260,34.0834,34.0315,34.0361,34.1098,34.1540,33.9368,0.0222,0.0314,0.0139,0.0264,0.0359,0.0211,0.0506,0.0239,0.0195,0.0244
2,2025-11-09,4,61,cos,36,0.3,1,34.52822,0.428319,0.07696,0.025820,34.8397,35.0390,35.1257,34.1310,34.6296,34.7337,34.0210,34.6563,34.0728,34.0334,0.1019,0.0576,0.1342,0.0958,0.0539,0.0594,0.0603,0.0749,0.0652,0.0664
3,2025-11-09,5,92,cos,36,0.3,3,33.41389,0.592093,0.13273,0.085554,33.9495,34.1444,33.5586,33.4852,32.9203,34.1612,33.1922,33.3552,33.1304,32.2419,0.0545,0.0252,0.0810,0.0973,0.1441,0.1443,0.1279,0.3019,0.2520,0.0991
4,2025-11-09,2,61,_cos,40,0.3,2,34.01628,0.334351,0.07227,0.023782,34.3612,34.4437,34.0495,33.9382,33.3032,34.1230,33.9392,33.9461,33.7411,34.3176,0.0815,0.0663,0.0519,0.0770,0.1233,0.0834,0.0848,0.0614,0.0567,0.0364
5,2025-11-09,1,61,_cos,40,0.4,2,34.09213,0.271552,0.04879,0.013215,34.5063,34.4195,34.0148,33.8809,33.9621,33.8735,33.8598,33.7669,34.3787,34.2588,0.0596,0.0394,0.0345,0.0579,0.0617,0.0406,0.0459,0.0678,0.0274,0.0531
6,2025-11-07,1,20,cos,48,0.4,2,34.03229,0.147213,0.04455,0.014940,33.8739,33.8561,34.0091,34.1713,34.1532,33.8691,34.1332,33.9426,34.0381,34.2763,0.0277,0.0379,0.0543,0.0614,0.0696,0.0501,0.0495,0.0227,0.0357,0.0366
7,2025-11-07,2,34,cos,48,0.4,2,34.20160,0.203500,0.03720,0.024600,33.8877,33.8170,34.3975,34.1070,34.3238,34.2922,34.3360,34.2902,34.3743,34.1906,0.0247,0.0238,0.0367,0.0367,0.0163,0.1037,0.0428,0.0314,0.0265,0.0291
8,2025-11-07,3,34,cos,48,0.4,2,33.77220,0.147600,0.09480,0.032400,33.7382,33.8192,33.5882,33.7218,33.4791,33.9450,33.9421,33.8468,33.8459,33.7960,0.1201,0.0930,0.0927,0.0939,0.0571,0.1116,0.1641,0.0793,0.0508,0.0850
9,2025-11-07,2,34,cos,60,0.3,3,34.16810,0.252500,0.05370,0.024600,33.9715,34.0325,34.5261,34.3204,34.4775,34.1714,34.3446,33.7550,33.9080,34.1736,0.0252,0.0654,0.0168,0.0602,0.0510,0.0906,0.0771,0.0743,0.0499,0.0266
